In [48]:
import gymnasium as gym
import numpy as np
import time
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions.categorical import Categorical
import matplotlib.pyplot as plt
from cellitaire.environment.cellitaire_env import CellitaireEnv
from cellitaire.environment.rewards.reward import *
from cellitaire.environment.rewards.foundation_rewards import *

In [49]:
class PPOMemory:
    def __init__(self, batch_size):
        self.states = []
        self.probs = []
        self.vals = []
        self.actions = []
        self.rewards = []
        self.dones = []

        self.batch_size = batch_size

    def generate_batches(self):
        n_states = len(self.states)
        batch_start = np.arange(0, n_states, self.batch_size)
        indices = np.arange(n_states, dtype=np.int64)
        np.random.shuffle(indices)
        batches = [indices[i:i+self.batch_size] for i in batch_start]

        return np.array(self.states), np.array(self.actions), np.array(self.probs), np.array(self.vals), np.array(self.rewards), np.array(self.dones), batches
        
    def store_memory(self, state, action, probs, vals, reward, done):
        self.states.append(state)
        self.probs.append(probs)
        self.vals.append(vals)
        self.rewards.append(reward)
        self.dones.append(done)
        self.actions.append(action)

    def clear_memory(self):
        self.states = []
        self.probs = []
        self.vals = []
        self.actions = []
        self.rewards = []
        self.dones = []

In [50]:
class ActorNetwork(nn.Module):
    def __init__(
        self, 
        n_actions, 
        input_dims, 
        alpha,
        chkpt_dir='tmp/ppo', 
        num_embeddings=53, 
        embedding_dim=30, 
        embeddings_in_state=85,
        num_hidden_layers=1,
        hidden_dim=256
    ):
        super(ActorNetwork, self).__init__()
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.embeddings_in_state = embeddings_in_state

        # Create embedding layer only if embeddings_in_state > 0.
        if embeddings_in_state > 0:
            self.embedding_layer = nn.Embedding(num_embeddings, embedding_dim).to(self.device)
            input_layer_dim = input_dims[0] - embeddings_in_state + (embeddings_in_state * embedding_dim)
        else:
            self.embedding_layer = None
            input_layer_dim = input_dims[0]

        self.checkpoint_file = os.path.join(chkpt_dir, 'actor_torch_ppo')
        
        # First layer with layer normalization and activation.
        self.actor = nn.Sequential(
            nn.Linear(input_layer_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU()
        )
        
        # Append hidden layers with layer normalization between linear and activation.
        for _ in range(num_hidden_layers):
            self.actor.append(nn.Linear(hidden_dim, hidden_dim))
            self.actor.append(nn.LayerNorm(hidden_dim))
            self.actor.append(nn.ReLU())
        
        # Final output layer.
        self.actor.append(nn.Linear(hidden_dim, n_actions))
        #self.actor.append(nn.Softmax(dim=-1))
        
        self.optimizer = optim.Adam(self.parameters(), lr=alpha)
        self.to(self.device)
        self.n_actions = n_actions

    def forward(self, state):
        if self.embeddings_in_state > 0:
            embeddings = self.embedding_layer(state[:, :self.embeddings_in_state].int()).view(state.shape[0], 1, -1)
            s = state.view(state.shape[0], 1, -1)[:, :, self.embeddings_in_state:]
            
            # Concatenate the remaining state features with the flattened embeddings.
            new_state = torch.cat((s, embeddings), dim=2)
        else:
            new_state = state.view(state.shape[0], 1, -1)
        
        x = new_state
        logits = self.actor(new_state)
        return Categorical(logits)

    def save_checkpoint(self):
        torch.save(self.state_dict(), self.checkpoint_file)

    def load_checkpoint(self):
        self.load_state_dict(torch.load(self.checkpoint_file))


In [51]:
class CriticNetwork(nn.Module):
    def __init__(
        self, 
        input_dims, 
        alpha, 
        num_hidden_layers=1, 
        hidden_dim=256, 
        chkpt_dir='tmp/ppo'
    ):
        super(CriticNetwork, self).__init__()

        self.checkpoint_file = os.path.join(chkpt_dir, 'critic_torch_ppo')

        self.critic = nn.Sequential(
            nn.Linear(*input_dims, hidden_dim),
            nn.ReLU()
        )
        
        for _ in range(num_hidden_layers):
            self.critic.append(nn.Linear(hidden_dim, hidden_dim))
            self.critic.append(nn.ReLU())

        self.critic.append(nn.Linear(hidden_dim, 1))

        self.optimizer = optim.Adam(self.parameters(), lr=alpha)
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.to(self.device)

    def forward(self, state):
        value = self.critic(state)
        return value

    def save_checkpoint(self):
        torch.save(self.state_dict(), self.checkpoint_file)

    def load_checkpoint(self):
        self.load_state_dict(torch.load(self.checkpoint_file))

In [57]:
class Agent:
    def __init__(
        self, 
        n_actions, 
        input_dims, 
        gamma=0.99, 
        alpha=0.0003, 
        gae_lambda=0.95,
        policy_clip=0.2, 
        batch_size=64, 
        n_epochs=10,
        num_hidden_layers_actor=1,
        hidden_dim_actor=256,
        num_hidden_layers_critic=1,
        hidden_dim_critic=256,
        embeddings_in_state_actor=85,
        embedding_dim_actor=30
    ):
        self.gamma = gamma
        self.policy_clip = policy_clip
        self.n_epochs = n_epochs
        self.gae_lambda = gae_lambda
        self.n_actions = n_actions

        self.actor = ActorNetwork(
            n_actions, 
            input_dims, 
            alpha, 
            num_hidden_layers=num_hidden_layers_actor, 
            hidden_dim=hidden_dim_actor,
            embeddings_in_state=embeddings_in_state_actor,
            embedding_dim=embedding_dim_actor
        )
        self.critic = CriticNetwork(
            input_dims, 
            alpha,
            num_hidden_layers=num_hidden_layers_critic,
            hidden_dim=hidden_dim_critic
        )
        actor_param_count = sum(p.numel() for p in self.actor.parameters())
        critic_param_count = sum(p.numel() for p in self.critic.parameters())
        print(f'NUM - PARAMS {actor_param_count + critic_param_count}')
        self.memory = PPOMemory(batch_size)
       
    def remember(self, state, action, probs, vals, reward, done):
        self.memory.store_memory(state, action, probs, vals, reward, done)

    def save_models(self):
        #print('... saving models ...')
        self.actor.save_checkpoint()
        self.critic.save_checkpoint()

    def load_models(self):
        #print('... loading models ...')
        self.actor.load_checkpoint()
        self.critic.load_checkpoint()
    
    def choose_action(self, observation, legal_actions=None):
        state = torch.tensor(np.array([observation]), dtype=torch.float).to(self.actor.device)
        '''
        # Get logits from the actor; assumed shape: (n, 1, k)
        logits = self.actor(state)

        # If legal_actions is provided, mask the logits to include only those actions.
        if legal_actions is not None:
            legal_actions = torch.tensor(np.array(legal_actions)).to(self.actor.device)
            masked_logits = logits[:, 0, legal_actions]  # shape: (n, len(legal_actions))
            dist = Categorical(logits=masked_logits)
        else:
            # Use all logits if no legal actions mask is provided.
            full_logits = logits.squeeze(1)  # shape: (n, k)
            dist = Categorical(logits=full_logits)
        '''

        dist = self.actor(state)

        value = self.critic(state)

        # Sample an action index from the distribution.
        sampled_index = dist.sample()  # shape: (n,)

        # Map back to the original action if a mask was applied.
        if legal_actions is not None:
            action = legal_actions[sampled_index]
        else:
            action = sampled_index

        # Get the log probability of the sampled action.
        log_prob = torch.squeeze(dist.log_prob(sampled_index)).item()

        # Squeeze and convert to Python scalars.
        action = torch.squeeze(action).item()
        value = torch.squeeze(value).item()

        return action, log_prob, value


    def learn(self):
        device = self.actor.device  # Assuming this is a CUDA device
        for _ in range(self.n_epochs):
            # Retrieve batch data
            state_arr, action_arr, old_prob_arr, vals_arr, reward_arr, dones_arr, batches = \
                self.memory.generate_batches()
            
            # Convert arrays to torch tensors on GPU
            rewards = torch.tensor(reward_arr, dtype=torch.float32, device=device)
            values = torch.tensor(vals_arr, dtype=torch.float32, device=device)
            dones = torch.tensor(dones_arr, dtype=torch.float32, device=device)
            
            advantage = compute_advantage(values, dones, rewards, self.gamma, self.gae_lambda)
            
            # Loop over minibatches
            for batch in batches:
                states = torch.tensor(state_arr[batch], dtype=torch.float, device=device)
                old_probs = torch.tensor(old_prob_arr[batch], device=device)
                actions = torch.tensor(action_arr[batch], device=device)
    
                dist = self.actor(states)
                critic_value = self.critic(states).squeeze()
                
                # Calculate probability ratio and losses
                new_probs = dist.log_prob(actions)
                prob_ratio = new_probs.exp() / old_probs.exp()
                weighted_probs = advantage[batch] * prob_ratio
                weighted_clipped_probs = torch.clamp(prob_ratio, 1 - self.policy_clip, 1 + self.policy_clip) * advantage[batch]
                actor_loss = -torch.min(weighted_probs, weighted_clipped_probs).mean()
    
                returns = advantage[batch] + values[batch]
                critic_loss = (returns - critic_value).pow(2).mean()
    
                total_loss = actor_loss + 0.5 * critic_loss
                
                # Update the networks
                self.actor.optimizer.zero_grad()
                self.critic.optimizer.zero_grad()
                total_loss.backward()
                #torch.nn.utils.clip_grad_norm_(self.actor.parameters(), max_norm=1)
                #torch.nn.utils.clip_grad_norm_(self.critic.parameters(), max_norm=1)
                self.actor.optimizer.step()
                self.critic.optimizer.step()
    
        self.memory.clear_memory()             

def compute_advantage(values, dones, rewards, gamma, gae_lambda):
    advantage = torch.zeros_like(rewards, device=values.device)
    T = len(rewards) - 1
    discount = gamma * gae_lambda
    discount_factors = discount ** torch.arange(T + 1, device=values.device, dtype=values.dtype)
    for t in range(T):
        discount_factors = discount_factors[:-1]
        next_vs = values[t+1:]
        masked = (1 - dones[t:-1]) * next_vs
        discounted_mask = masked * gamma
        discounted_mask_plus_rewards = rewards[t:-1] + discounted_mask
        diff = discounted_mask_plus_rewards - values[t:-1]
        discounted_diff = discount_factors * diff
        advantage[t] = discounted_diff.sum()
    return advantage

In [6]:
torch.cuda.is_available()

True

In [7]:
board_rows = 7
board_cols = 12
num_reserved = 6
test_reward = CombinedReward([
    #PlacedCardInFoundationReward(weight=6),
    WinReward(rows=board_rows, cols=board_cols),
    ConstantReward(weight=0.5),
    ScalingPlacedCardInFoundationReward(weight=1, rows=board_rows, cols=board_cols)
    #PlayedLegalMoveReward(weight=1),
    #PeriodicPlacedCardInFoundationReward(weight=4, reward_period=3),
    #CreatedMovesReward(weight=1, num_reserved=num_reserved, foundation_count_dropoff=30)
])
env = CellitaireEnv(test_reward, rows=board_rows, cols=board_cols, num_reserved=num_reserved, max_moves=600)

[1000000, 0.5, 1]


In [8]:
torch.autograd.set_detect_anomaly(False)
N = 60000
batch_size = 3000
n_epochs = 15
alpha = 1e-6
num_hidden_layers_actor=2
hidden_dim_actor=4096
num_hidden_layers_critic=4
hidden_dim_critic=4096
#embeddings_in_state_actor=1
embeddings_in_state_actor = board_rows * board_cols + 1
embedding_dim=30

env.reset()

agent = Agent(
    
    n_actions=env.action_space.n, 
    input_dims=(board_rows * board_cols * 4 + 6,), 
    batch_size=batch_size, 
    alpha=alpha, 
    n_epochs=n_epochs,
    num_hidden_layers_actor=num_hidden_layers_actor,
    hidden_dim_actor=hidden_dim_actor,
    embeddings_in_state_actor=embeddings_in_state_actor,
    embedding_dim_actor=embedding_dim,
    num_hidden_layers_critic=num_hidden_layers_critic,
    hidden_dim_critic=hidden_dim_critic,
)

n_games = 1

best_score = -1000
score_history = []
moves_history = []
cards_saved_history = []
episodes_without_best = 0

learn_iters = 0
avg_score = 0
avg_moves = 0
avg_cards_saved = 0
n_steps = 0

i = 0
while True:
    observation, reward, done, truncated, info = env.reset()
    observation = env.get_state()
    done = False
    score = 0
    while (not done) and (not truncated):
        legal_actions = env.get_legal_actions_as_int()
        action, prob, val = agent.choose_action(observation, legal_actions)
        observation_, reward, done, truncated, info = env.step(action)
        n_steps += 1
        score += reward
        agent.remember(observation, action, prob, val, reward, done)
        if n_steps % N == 0:
            agent.learn()
            learn_iters += 1
        observation = observation_
    score_history.append(score)
    moves_history.append(env.num_moves)
    cards_saved_history.append(env.game.foundation.total_cards())
    avg_score = np.mean(score_history[-100:])
    avg_moves = np.mean(moves_history[-100:])
    avg_cards_saved = np.mean(cards_saved_history[-100:])
    i += 1

    if avg_score > best_score and n_steps > N:
        best_score = avg_score
        #agent.save_models()
        episodes_without_best = 0
        print(f'episode {i:>5} | score {score:>6.1f} | recent avg {avg_score:>6.1f} | avg moves {avg_moves:>5.1f} | avg saved {avg_cards_saved:>4.1f} | learning steps {learn_iters:6} | done {done} *') 
    else:
        episodes_without_best += 1
        
    if episodes_without_best % 100 == 0 and episodes_without_best > 0:
        print(f'episode {i:>5} | score {score:>6.1f} | recent avg {avg_score:>6.1f} | avg moves {avg_moves:>5.1f} | avg saved {avg_cards_saved:>4.1f} | learning steps {learn_iters:6} | done {done}') 

<generator object Module.parameters at 0x00000203A4089380>
NUM - PARAMS 113968779


KeyboardInterrupt: 

In [ ]:
plt.plot(torch.tensor(score_history, dtype=torch.float).view(-1, 2000).mean(1))

In [ ]:
plt.plot(torch.tensor(moves_history, dtype=torch.float).view(-1, 2000).mean(1))

In [ ]:
plt.plot(torch.tensor(cards_saved_history, dtype=torch.float).view(-1, 2000).mean(1))

In [ ]:
print(max(cards_saved_history))

In [53]:
env = gym.make("CartPole-v1", render_mode='human')
env.reset()
env.render()

In [60]:
#N = 20
#batch_size = 5
#n_epochs = 4
#alpha = 0.0003
#embeddings_in_state_actor=1

N = 20
batch_size = 5
n_epochs = 4
alpha = 0.0003
num_hidden_layers_actor=1
hidden_dim_actor=256
embeddings_in_state_actor=0
#embeddings_in_state_actor = board_rows * board_cols + 1
embedding_dim=30

'''
agent = Agent(
    n_actions=env.action_space.n, 
    input_dims=env.observation_space.shape, 
    batch_size=batch_size, 
    alpha=alpha, 
    n_epochs=n_epochs,
    embeddings_in_state_actor=embeddings_in_state_actor
)
'''

agent = Agent(
    
    n_actions=env.action_space.n, 
    input_dims=env.observation_space.shape, 
    batch_size=batch_size, 
    alpha=alpha, 
    n_epochs=n_epochs,
    num_hidden_layers_actor=num_hidden_layers_actor,
    hidden_dim_actor=hidden_dim_actor,
    embeddings_in_state_actor=embeddings_in_state_actor,
    embedding_dim_actor=embedding_dim
)

n_games = 1500

best_score = -1000
score_history = []
max_score = 0
episodes_without_best = 0

learn_iters = 0
avg_score = 0
n_steps = 0

for i in range(n_games):
    (observation, _) = env.reset()
    done = False
    truncated = False
    score = 0
    while not done and not truncated:
        action, prob, val = agent.choose_action(observation)
        observation_, reward, done, truncated, info = env.step(action)
        n_steps += 1
        score += reward
        agent.remember(observation, action, prob, val, reward, done)
        if n_steps % N == 0:
            agent.learn()
            learn_iters += 1
        observation = observation_
    score_history.append(score)
    avg_score = np.mean(score_history[-100:])
    
    max_score = max(max_score, score)
    
    if avg_score > best_score:
        best_score = avg_score
        recent_std = np.std(score_history[-100:])
        #agent.save_models()
        episodes_without_best = 0
        print(f'episode {i:>5} | score {score:>6.1f} | avg {avg_score:>6.1f} | std {recent_std:>6.2f} | max score {max_score:>5.1f} | learning steps {learn_iters:>5} | done {done} *')
    else:
        episodes_without_best += 1
        
    if episodes_without_best % 100 == 0 and episodes_without_best > 0:
        recent_std = np.std(score_history[-100:])
        print(f'episode {i:>5} | score {score:>6.1f} | avg {avg_score:>6.1f} | std {recent_std:>6.2f} | max score {max_score:>5.1f} | learning steps {learn_iters:>5} | done {done}')
    

plt.plot(x, score_history)
env.close()

NUM - PARAMS 135939


ValueError: Expected parameter probs (Tensor of shape (1, 1, 2)) of distribution Categorical(probs: torch.Size([1, 1, 2])) to satisfy the constraint Simplex(), but found invalid values:
tensor([[[ 3.6367, -2.6367]]], device='cuda:0', grad_fn=<DivBackward0>)

In [61]:
env.close()

In [ ]:
%pip install pygame